In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [146]:
#chargement des nomenclatures
libelles = pd.read_csv('libelles.csv', sep=';', index_col=['colonne', 'indice'])
#libelles = libelles.set_index(['colonne', 'indice'])

variables = pd.read_csv('variables.csv', sep=';', index_col = ['variable'])


In [167]:
#chargement des données
annees = [str(x) for x in np.arange(2019,2023)]
#fichiers = ['caracteristiques', 'lieux', 'usagers','vehicules']
carac = pd.DataFrame()
lieux = pd.DataFrame()
usagers = pd.DataFrame()
vehicules = pd.DataFrame()
for annee in annees : 
    carac = pd.concat([carac, pd.read_csv(f"caracteristiques-{annee}.csv", sep=';')])
    lieux = pd.concat([lieux, pd.read_csv(f"lieux-{annee}.csv", sep=';')])
    usagers = pd.concat([usagers, pd.read_csv(f"usagers-{annee}.csv", sep=';')])
    vehicules = pd.concat([vehicules, pd.read_csv(f"vehicules-{annee}.csv", sep=';')])


C:\Users\Eric\AppData\Local\Temp\ipykernel_2472\1100335970.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.concat([lieux, pd.read_csv(f"lieux-{annee}.csv", sep=';')])


In [168]:
#correction identifiant accident
carac['Num_Acc'].fillna(carac['Accident_Id'], inplace=True)
carac = carac.drop('Accident_Id', axis = 1)

#jointure carac-lieux
df = pd.merge(left = carac, right = lieux, left_on = 'Num_Acc', right_on='Num_Acc')

#jointure df-vehicules
df = pd.merge(left=df, right=vehicules, left_on='Num_Acc', right_on='Num_Acc')

#jointure df-usagers
df = pd.merge(left=df, right=usagers, right_on='id_vehicule', left_on='id_vehicule')

#suppression des identifiants
df = df.drop(['Num_Acc_x', 'id_vehicule', 'num_veh_x', 'Num_Acc_y','num_veh_y', 'id_usager'], axis = 1)

In [197]:
df = df.replace({
    '-1':np.nan,
    -1:np.nan,
    ' -1':np.nan,
    '#ERREUR':np.nan
})



In [170]:
#secu  : un champ binaire par équipement
for i in range(1,10) :
    df['secu_'+str(i)] = ((df['secu1']==i) | (df['secu2']==i) | (df['secu3']==i))*1

df = df.drop(['secu1','secu2','secu3'], axis = 1)

In [171]:
pd.DataFrame({'column_name': df.columns,'percent_missing': df.isnull().sum() * 100 / len(df)}).sort_values('percent_missing')


,column_name,percent_missing
jour,jour,0.000000
secu_7,secu_7,0.000000
secu_6,secu_6,0.000000
secu_5,secu_5,0.000000
secu_4,secu_4,0.000000
secu_3,secu_3,0.000000
secu_2,secu_2,0.000000
secu_1,secu_1,0.000000
catu,catu,0.000000
secu_8,secu_8,0.000000


In [172]:
#elimination des colonnes avec plus de 25% de valeurs manquantes
df.dropna(thresh=len(df)*3/4, axis = 1, inplace= True)

In [173]:
#suppression des lignes sans variable d'Interet
df.dropna(subset=['grav'], inplace = True)

In [174]:
# suppression de l'adresse qui ne sert à rien
df = df.drop('adr', axis = 1)

In [175]:
df.drop(['voie','v1'], axis = 1, inplace=True)

In [176]:
df.dropna(how = 'any', axis=0, inplace=True)

In [177]:
df.isna().sum()

jour       0
mois       0
an         0
hrmn       0
lum        0
dep        0
com        0
agg        0
int        0
atm        0
col        0
lat        0
long       0
catr       0
circ       0
nbv        0
vosp       0
prof       0
pr         0
pr1        0
plan       0
surf       0
infra      0
situ       0
vma        0
senc       0
catv       0
obs        0
obsm       0
choc       0
manv       0
motor      0
place      0
catu       0
grav       0
sexe       0
an_nais    0
trajet     0
secu_1     0
secu_2     0
secu_3     0
secu_4     0
secu_5     0
secu_6     0
secu_7     0
secu_8     0
secu_9     0
dtype: int64

In [178]:
df.shape

(442991, 47)

In [179]:
#conversion des latitudes : longitudes
df['lat']= df['lat'].str.replace(',','.').astype('float')
df['long']= df['long'].str.replace(',','.').astype('float')


In [180]:
#on ajoute un créneau horaire
df['hh'] = df['hrmn'].str[:2]

df.drop('hrmn', axis = 1, inplace= True)

In [181]:
df['hh'] = df['hh'].astype('int')

In [182]:
#on ajoute la date  partir de jour, mois, année
df['date'] = pd.to_datetime(dict(day=df['jour'], month=df['mois'], year=df['an']))

In [183]:
#calcul de l'age des usagers dans l'année de l'accident
df['age'] = df['an'] - df['an_nais'].astype(int)

df.drop('an_nais', axis = 1, inplace = True)

In [184]:
df.drop(['jour','an'], axis = 1, inplace=True)

In [185]:
df.drop(['pr','pr1'], axis = 1, inplace=True)

In [186]:
df.drop(['senc'], axis = 1, inplace=True)

In [187]:
df.drop(['dep','com'], axis = 1, inplace=True)

In [215]:
df.shape

(442991, 41)

In [204]:
df.select_dtypes('float').columns

Index(['lat', 'long'], dtype='object')

In [202]:
df[['lum', 'int', 'atm', 'col',  'circ', 'vosp', 'prof',
       'plan', 'surf', 'infra', 'situ', 'vma', 'catv', 'obs', 'obsm', 'choc',
       'manv', 'motor', 'place', 'grav', 'sexe', 'trajet']] = df[['lum', 'int', 'atm', 'col',  'circ', 'vosp', 'prof',
       'plan', 'surf', 'infra', 'situ', 'vma', 'catv', 'obs', 'obsm', 'choc',
       'manv', 'motor', 'place', 'grav', 'sexe', 'trajet']].astype('int')

In [219]:
df.duplicated().sum()

459

In [220]:
for v in df.columns :
    cont = pd.crosstab(df[v],df['grav'])
    print(v, ' : ' , chi2_contingency(cont).pvalue)

mois  :  3.7166259163782275e-90
lum  :  0.0
agg  :  0.0
int  :  0.0
atm  :  7.903629026852421e-306
col  :  0.0
lat  :  0.0
long  :  0.0
catr  :  0.0
circ  :  0.0
nbv  :  0.0
vosp  :  8.268260976663102e-258
prof  :  0.0
plan  :  0.0
surf  :  1.5599881238016478e-174
infra  :  5.942802388227896e-197
situ  :  0.0
vma  :  0.0
catv  :  0.0
obs  :  0.0
obsm  :  0.0
choc  :  0.0
manv  :  0.0
motor  :  0.0
place  :  0.0
catu  :  0.0
grav  :  0.0
sexe  :  0.0
trajet  :  0.0
secu_1  :  0.0
secu_2  :  0.0
secu_3  :  1.0816435368773264e-37
secu_4  :  5.128152597204853e-83
secu_5  :  1.1022350030952272e-35
secu_6  :  0.0
secu_7  :  4.974363745844205e-23
secu_8  :  0.0
secu_9  :  0.0
hh  :  0.0
date  :  2.6796861381686795e-140
age  :  0.0
